In [144]:
from tempfile import TemporaryDirectory
from pathlib import Path
import subprocess

import numpy as np
import tomlkit
import awkward as ak
import pandas as pd
import yaml
from IPython.display import display

In [241]:
with open('../configs/default.toml', 'r') as fin:
    config = tomlkit.load(fin)
VOTING = '../target/release/voting'

In [221]:
def do_run(config, trials) -> ak.Array:
    with TemporaryDirectory() as temp_dir:
        temp_dir = Path(temp_dir)
        config_file = temp_dir / 'config.toml'
        with open(config_file, 'w') as fout:
            tomlkit.dump(config, fout)
        pq_file = temp_dir / 'results.parquet'
        cp = subprocess.run([
            VOTING,
            '-c', config_file,
            '-o', pq_file,
            '-t', str(trials),
        ], check=True, capture_output=True)
        if cp.returncode != 0:
            print(cp.stderr)
            return None
        results = ak.from_parquet(pq_file)
    return results

In [206]:
def get_fields(results):
    return list(filter(lambda fn: fn.startswith('m:'), results.fields))     

In [224]:
def run_experiment(config, parameter, pvalues, trials=2000):
    fields = None
    col_names = None
    regrets = []
    parameters = [int(p) if p.isnumeric() else p for p in parameter.split('.')]
    last_parameter = parameters.pop()
    for val in pvalues:
        conf_item = config
        for p in parameters:  # drill down into config
            conf_item = conf_item[p]
        conf_item[last_parameter] = val
        results = do_run(config, trials)
        if fields is None:
            fields = get_fields(results)
            col_names = [f[2:] for f in fields]
        regrets.append([val] + [ak.mean(results[f]['regret']) for f in fields])
    return pd.DataFrame(regrets, columns=[parameter] + col_names)

In [225]:
config['voters'] = 333
df = run_experiment(config, 'candidates', [2, 3, 4, 5, 6])
df

,candidates,pl_h,pl_s,range_10_h,range_10_s,aprv_h,aprv_s,IRV_h,Borda_h,multi_h_3v
0,2,0.174000,0.174000,0.174000,0.174000,0.174000,0.174000,0.174000,0.174000,0.174000
1,3,0.578939,0.219201,0.105181,0.070680,0.089072,0.084015,0.218270,0.167260,0.335496
2,4,0.634941,0.244819,0.072011,0.049376,0.079200,0.060734,0.253010,0.129646,0.207927
3,5,0.671219,0.275642,0.060983,0.047957,0.080293,0.055073,0.279514,0.114746,0.286294
4,6,0.659681,0.316442,0.051909,0.049277,0.088585,0.057836,0.307957,0.105407,0.171682


In [236]:
config['candidates'] = 4
config['voters'] = 10000
config['considerations'] = yaml.safe_load(
'''
- Likability:
    mean: 1.0
'''
)
result = do_run(config, 5)
display(result['cand_regret'][0])
display(result['cov_matrix'][0])

<Array [0, 0.00892, 1.99, 2] type='4 * ?float64'>

<Array [[0], [0, 0], [...], [0, 0, 0, 0]] type='4 * option[var * ?float64]'>

In [228]:
def get_corr_coeffs(result, trial_no):
    covm = result['cov_matrix'][trial_no]
    ncand = len(covm)
    cc = np.eye(ncand, dtype=np.float64)
    sigmas = []
    for i in range(ncand):
        sigmas.append(np.sqrt(covm[(i, i)]))
    print(sigmas)
    for ix in range(ncand):
        for iy in range(ix):
            cc[(ix, iy)] = covm[(ix, iy)] / sigmas[ix] / sigmas[iy]
    return cc

In [231]:
display(get_corr_coeffs(result, 0))

[np.float64(2.110951164381278), np.float64(1.725697028135377), np.float64(1.7006315355210861), np.float64(1.4473983208331664)]


array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [-0.91255087,  1.        ,  0.        ,  0.        ],
       [ 0.92302302, -0.7848075 ,  1.        ,  0.        ],
       [-0.81113237,  0.9651004 , -0.63174041,  1.        ]])

In [235]:
config['candidates'] = 4
config['voters'] = 10000
config['considerations'] = yaml.safe_load(
'''
- Irrational:
    sigma: 1.0
    camps: 2
    individualism_deg: 90
'''
)
result = do_run(config, 5)
display(get_corr_coeffs(result, 0))
display(get_corr_coeffs(result, 1))
display(get_corr_coeffs(result, 2))
display(get_corr_coeffs(result, 3))

[np.float64(0.9948045570651918), np.float64(1.0011200090887933), np.float64(1.0014856574876458), np.float64(1.004928151257509)]


array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [-0.01098302,  1.        ,  0.        ,  0.        ],
       [-0.0057799 ,  0.00444545,  1.        ,  0.        ],
       [ 0.0056485 , -0.01203408,  0.01468783,  1.        ]])

[np.float64(1.0065255493621288), np.float64(0.998540921410261), np.float64(1.0020417678612101), np.float64(0.9982811889837808)]


array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.02094225,  1.        ,  0.        ,  0.        ],
       [ 0.01240603,  0.00624386,  1.        ,  0.        ],
       [-0.003224  , -0.00306613, -0.00120506,  1.        ]])

[np.float64(0.9964617508629647), np.float64(0.9966730766811684), np.float64(1.0014015346113174), np.float64(1.011845926550513)]


array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [-0.01173113,  1.        ,  0.        ,  0.        ],
       [ 0.00300255,  0.01332203,  1.        ,  0.        ],
       [ 0.00422992,  0.00458867, -0.02422909,  1.        ]])

[np.float64(1.0028046663451333), np.float64(0.9973422849777929), np.float64(1.001718554467597), np.float64(1.001001815016451)]


array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.96392678e-02,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 2.20155134e-03, -5.74468956e-04,  1.00000000e+00,
         0.00000000e+00],
       [-1.69129125e-03,  1.47832724e-02, -3.48291790e-03,
         1.00000000e+00]])

In [200]:
utilities = np.random.uniform(0., np.sqrt(12.), 50)
utilities = np.reshape(utilities, newshape=(5, 10))
utilities

array([[0.3090016 , 2.09904798, 0.82317008, 3.13087807, 1.61064933,
        2.01657254, 2.91759999, 1.55519431, 2.89235991, 2.97551052],
       [2.24721985, 2.65914812, 1.86329343, 2.21673783, 0.84797068,
        0.56396652, 0.44596168, 3.40639598, 0.65299382, 1.77379581],
       [0.58539738, 1.22377722, 3.06071522, 1.3184892 , 0.48842566,
        0.79741721, 2.61924207, 3.07438678, 1.22978133, 3.12792367],
       [2.85872826, 0.52028883, 1.38963001, 0.35922687, 0.16900879,
        1.34662276, 0.17156912, 1.73254542, 1.48895542, 1.27941031],
       [0.78623712, 0.66519608, 1.30919134, 2.73176796, 1.99823116,
        3.12736477, 3.38542127, 1.83569564, 3.25960743, 1.54455224]])

In [201]:
np.set_printoptions(threshold=100000)
print(str(utilities.transpose()))

[[0.3090016  2.24721985 0.58539738 2.85872826 0.78623712]
 [2.09904798 2.65914812 1.22377722 0.52028883 0.66519608]
 [0.82317008 1.86329343 3.06071522 1.38963001 1.30919134]
 [3.13087807 2.21673783 1.3184892  0.35922687 2.73176796]
 [1.61064933 0.84797068 0.48842566 0.16900879 1.99823116]
 [2.01657254 0.56396652 0.79741721 1.34662276 3.12736477]
 [2.91759999 0.44596168 2.61924207 0.17156912 3.38542127]
 [1.55519431 3.40639598 3.07438678 1.73254542 1.83569564]
 [2.89235991 0.65299382 1.22978133 1.48895542 3.25960743]
 [2.97551052 1.77379581 3.12792367 1.27941031 1.54455224]]


In [202]:
np.cov(utilities)

array([[ 0.94020538, -0.32073684,  0.16108757, -0.49611513,  0.62033183],
       [-0.32073684,  1.01169877,  0.31368278,  0.27153381, -0.70086438],
       [ 0.16108757,  0.31368278,  1.19057637, -0.00328159, -0.05275154],
       [-0.49611513,  0.27153381, -0.00328159,  0.71388613, -0.31396175],
       [ 0.62033183, -0.70086438, -0.05275154, -0.31396175,  1.02640331]])

In [220]:
utilities = np.random.uniform(0., np.sqrt(12.), 5000)
utilities = np.reshape(utilities, newshape=(5, 1000))
np.cov(utilities)

array([[ 0.99333244, -0.00444264,  0.01690256,  0.0234099 , -0.03744667],
       [-0.00444264,  1.01777285, -0.02395978, -0.02019903, -0.06810528],
       [ 0.01690256, -0.02395978,  0.99728841, -0.08037311, -0.03790731],
       [ 0.0234099 , -0.02019903, -0.08037311,  1.00854158,  0.02726133],
       [-0.03744667, -0.06810528, -0.03790731,  0.02726133,  0.99367376]])

In [277]:
config['candidates'] = 4
config['voters'] = 10000
config['considerations'] = yaml.safe_load(
'''
- Issues:
    issues:
    - halfcsep: 1.0
      halfvsep: 1.0
      sigma: 1.0
'''
)
result = do_run(config, 5)
display(get_corr_coeffs(result, 0))
display(get_corr_coeffs(result, 1))
display(get_corr_coeffs(result, 2))
display(get_corr_coeffs(result, 3))

[np.float64(1.1287110091352806), np.float64(1.032109855620688), np.float64(1.3461672012653416), np.float64(0.9533230294679127)]


array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [-0.54892674,  1.        ,  0.        ,  0.        ],
       [ 0.92176246, -0.75311506,  1.        ,  0.        ],
       [-0.40796007,  0.98008771, -0.63788827,  1.        ]])

[np.float64(0.819918112352644), np.float64(0.9482917975727155), np.float64(0.8641959032228944), np.float64(0.8449585108798815)]


array([[1.        , 0.        , 0.        , 0.        ],
       [0.85582997, 1.        , 0.        , 0.        ],
       [0.60756034, 0.15686375, 1.        , 0.        ],
       [0.6748163 , 0.23935748, 0.99530548, 1.        ]])

[np.float64(0.8422491927110479), np.float64(1.047319651610908), np.float64(1.1836007981186247), np.float64(1.174304279588924)]


array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.01355423,  1.        ,  0.        ,  0.        ],
       [ 0.67120621, -0.65080222,  1.        ,  0.        ],
       [ 0.68166488, -0.64141072,  0.99982942,  1.        ]])

[np.float64(0.872599017437322), np.float64(0.8003349482034474), np.float64(1.110727742826908), np.float64(0.974612006112627)]


array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.62130842,  1.        ,  0.        ,  0.        ],
       [ 0.82984412,  0.14063478,  1.        ,  0.        ],
       [ 0.020431  ,  0.75907463, -0.46486468,  1.        ]])

In [275]:
print(yaml.safe_dump(config.unwrap()))

candidates: 4
considerations:
- Likability:
    mean: 0.1
- Issues:
    issues:
    - halfcsep: 2.0
      halfvsep: 2.0
      sigma: 1.0
    - halfcsep: 0.75
      halfvsep: 0.75
      sigma: 0.5
methods:
- Plurality:
    strat: Honest
- Plurality:
    strat: Strategic
- Range:
    nranks: 10
    strat: Honest
- Range:
    nranks: 10
    strat: Strategic
- Range:
    nranks: 2
    strat: Honest
- Range:
    nranks: 2
    strat: Strategic
- InstantRunoff: {}
- Borda: {}
- Multivote:
    spread_fact: 1.0
    strat: Honest
    votes: 3
primary_method:
  RRV:
    k: 0.5
    ranks: 21
    strat: Honest
voters: 101

